<a href="https://colab.research.google.com/github/Raju1410/Class_IS532E/blob/main/Session3/RAG_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langchain langchain-community
!pip install faiss-gpu
!pip install sentence_transformers
!pip install pypdf
!pip install bitsandbytes transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 523.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.

## Downloading the pdf file

In [2]:
import requests

# URL of the PDF file
url = "https://web-assets.bcg.com/img-src/Reshaping%20Business%20with%20Artificial%20Intelligence_tcm9-177882.pdf"

# Send a GET request to download the PDF
response = requests.get(url)

# Save the PDF to a file
with open("ai_business_bcg.pdf", "wb") as file:
    file.write(response.content)

print("PDF downloaded successfully!")

PDF downloaded successfully!


## RAG Workflow
![RAG Workflow](https://cdn-lfs-us-1.hf.co/repos/13/3d/133d8ca2460bf82ba2bdbe928d91a6c780364a6d0cf9005087db081cca492c02/ed22547b1538ea4fd18ea26777e14d9f7e51b3388b34d3cadf165cc37a7f63e0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27RAG_workflow.png%3B+filename%3D%22RAG_workflow.png%22%3B&response-content-type=image%2Fpng&Expires=1728491852&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODQ5MTg1Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzEzLzNkLzEzM2Q4Y2EyNDYwYmY4MmJhMmJkYmU5MjhkOTFhNmM3ODAzNjRhNmQwY2Y5MDA1MDg3ZGIwODFjY2E0OTJjMDIvZWQyMjU0N2IxNTM4ZWE0ZmQxOGVhMjY3NzdlMTRkOWY3ZTUxYjMzODhiMzRkM2NhZGYxNjVjYzM3YTdmNjNlMD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV19&Signature=CxEoLq287jIE6qsl8i1VcLfERGZxQ5knn1vhdISqZCUOs8yNcRUwGYxQBfEd4fYpVG7W0lI6PnXNIZROylpuVd4qWnJRwZwS-wGLM3BtCIP2A37w%7E%7E30Onm5dnAP7yXGsGJfW14M%7EuAzr1igz2oW9uyNttaHZDarPnw54ogegGmIrQYUurnMBnoG-Wqlv%7ENn5YDgDOOXE2cXTwzuQVnXRvwqsHdISWatnQH571L%7EpvJut3KdWD5969FOx5arC2DMVkzqlW8Mjlh7ZBAUCJ8nQdoEgKrhOZb1MmQ65twSYdwEw9zROWP2QYUE%7E-eXNLfAASc6UfrqlEXkwbUTV4ztMg__&Key-Pair-Id=K24J24Z295AEI9)

**Source**:https://huggingface.co/learn/cookbook/en/advanced_rag


In [3]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF
loader = PyPDFLoader("/content/ai_business_bcg.pdf")
pages = loader.load_and_split()

# Check the number of pages loaded
print(f"Number of pages loaded: {len(pages)}")


Number of pages loaded: 28


In [4]:
pages[0]

Document(metadata={'source': '/content/ai_business_bcg.pdf', 'page': 0}, page_content='FINDINGS FROM THE 2017 ARTIFICIAL INTELLIGENCE  \nGLOBAL EXECUTIVE STUDY AND RESEARCH PROJECT\n#MITSMRREPORT\nREPRINT NUMBER 59181         \n13%\nNeithe r83% \nSTRA TEGIC OPPO RTUNITY\n4%\nRisk\nonly\n50%\nOppor tunity \nonly33%\nBoth oppor tunity \nand risk37%\nSTRA TEGIC  \nRISK4\nAI as strate gic oppor tunity and risk\nDo you percei ve AI as a strategic oppo rtunity or risk to your organiz ation?\nReshaping \nBusiness With Artificial Intelligence\nClosing the Gap Between  \nAmbition and Action\nFALL 2017RESEARCH \nREPORT\nBy Sam Ransbotham, David Kiron, Philipp Gerbert, \nand Martin ReevesIn collaboration with')

![RAG Indexing](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

*Source: Langchain Documentation*

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Step 3: Initialize RecursiveCharacterTextSplitter with desired chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Step 4: Apply the text splitter to split the entire document (all pages)
docs = text_splitter.split_documents(pages)

# Step 5: Display all chunks (you can customize this to work with the chunks as needed)
for i, doc in enumerate(docs):
    print(f"Chunk {i+1}:\n{doc.page_content}\n")

## Convert the chuncks into embedding and store them in a vector database

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Create a FAISS vector store from documents using Hugging Face embeddings
db = FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))

<ipython-input-6-02a86c0ddc96>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  db = FAISS.from_documents(docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a t

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Specify the model name
model_name = "HuggingFaceH4/zephyr-7b-beta"

# Configure BitsAndBytes for 4-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model with the specified configuration
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

![RAG Retrieval Generation](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)


In [10]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

# Create a text generation pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

# Initialize the HuggingFacePipeline with the text generation pipeline
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Define the prompt template
prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """
# Create a PromptTemplate instance
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create an LLM chain
llm_chain = prompt | llm | StrOutputParser()

<ipython-input-10-acd1a4bd1d01>:19: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [11]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [12]:
question = "Explain the barriers of AI adoption"

In [13]:
output = llm_chain.invoke({"context": "", "question": question})
output

'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n\n\n</s>\n<|user|>\nExplain the barriers of AI adoption\n</s>\n<|assistant|>\n\n  While artificial intelligence (AI) has the potential to revolutionize various industries and improve efficiency, there are several barriers that hinder its widespread adoption. Some of these barriers include:\n\n1. Data availability and quality: AI algorithms require large amounts of high-quality data to learn and make accurate predictions. However, many organizations lack the necessary data or have data that is not clean, consistent, or relevant enough for training AI models. This can result in poor performance and decreased trust in AI systems.\n\n2. Lack of expertise: Implementing AI requires a significant amount of technical expertise, which is often in short supply. Many organizations do not have the necessary skills in-house to develop, deploy, and maintain AI systems. This can lead to high costs associ

In [15]:
from pprint import pprint
pprint(output.replace('\n', ' ').strip())

('<|system|> Answer the question based on your knowledge. Use the following '
 'context to help:    </s> <|user|> Explain the barriers of AI adoption </s> '
 '<|assistant|>    While artificial intelligence (AI) has the potential to '
 'revolutionize various industries and improve efficiency, there are several '
 'barriers that hinder its widespread adoption. Some of these barriers '
 'include:  1. Data availability and quality: AI algorithms require large '
 'amounts of high-quality data to learn and make accurate predictions. '
 'However, many organizations lack the necessary data or have data that is not '
 'clean, consistent, or relevant enough for training AI models. This can '
 'result in poor performance and decreased trust in AI systems.  2. Lack of '
 'expertise: Implementing AI requires a significant amount of technical '
 'expertise, which is often in short supply. Many organizations do not have '
 'the necessary skills in-house to develop, deploy, and maintain AI systems. '
